***IMPORTS Y ACCESO A DATOS DE CSV BICIS***

In [289]:
#Importación de paquetes y lectura de csv:
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import pandas as pd
import numpy as np
import random
from sklearn import model_selection
from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr
import xgboost as xgb
from xgboost import DMatrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import copy

random.seed(27)
dataBicis = pd.read_csv('hour.csv')
dataBicis = dataBicis.drop("dteday", axis=1)
dataBicis = dataBicis.drop("instant", axis=1)
atributosBicis = dataBicis.loc[:, 'season':'registered']
objetivoBicis = dataBicis['cnt']

(atributos_entrenamientoBicis, atributos_pruebaBicis,
 objetivo_entrenamientoBicis, objetivo_pruebaBicis) = model_selection.train_test_split(
    atributosBicis, objetivoBicis, test_size=.33)

atributos_entrenamientoBicis = atributos_entrenamientoBicis.values
objetivo_entrenamientoBicis = objetivo_entrenamientoBicis.values
atributos_pruebaBicis = atributos_pruebaBicis.values
objetivo_pruebaBicis= objetivo_pruebaBicis.values

rangosBicis = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in dataBicis.columns:
    valor_min = dataBicis[columna].min()
    valor_max = dataBicis[columna].max()
    rangosBicis[columna] = {'min': valor_min, 'max': valor_max}

***MÉTODO LIME***

In [232]:
def metodo_lime(N,x,k,model,rangos):
    random.seed(1)
    Xprima=[]
    R=[]
    W=[]
    Yprima= []
    for i in range (0,N):
        atributosAPerturbar=seleccionar_atributos_aleatorios(x,k)
        xprima = perturbar_muestra(x,atributosAPerturbar,rangos)
        w = distancia_entre_muestras(x,xprima)
        r=atributosAPerturbar
        Xprima.append(xprima)
        R.append(r)
        W.append(w)
        yprima = prediccion_de_modelo(xprima,model)
        Yprima.append(yprima)
    W=np.array(W)
    W = W.reshape(W.shape[0])
    G = entrenar_modelo_ridge(R,Yprima,W)
    return G.coef_

#Se seleccional los k atributos aleatorios a modificar

def seleccionar_atributos_aleatorios(dataframe, k):
    columnas = dataframe.columns.tolist()
    seleccionados = random.sample(range(len(columnas)), k)
    lista = [0] * len(columnas)
    for i in seleccionados:
        lista[i] = 1
    #print(lista)
    return lista

#Se perturba la muestra original de forma que los atributos seleccionados pueden variar en un rango en función del valor máximo y mínimo de dicho atributo
def perturbar_muestra(x, atributosAPerturbar,rangos):
    xprima = x.copy()
    for i in range(0,len(atributosAPerturbar)):
        if atributosAPerturbar[i] == 1:
            rango_min = xprima[i] - rangos[list(rangos.keys())[i]]['min']
            rango_max = rangos[list(rangos.keys())[i]]['max'] - xprima[i]
            xprima[i] = random.uniform(rango_min, rango_max)
    return xprima

# Se calcula la distancia coseno entre la muestra original y la perturbada
def distancia_entre_muestras(x,xprima):
    return cosine_distances(x.values, xprima.values)

#Se predice el resultado de la muestra con el modelo elegido
def prediccion_de_modelo(xprima,model):
    if not isinstance(model, (RandomForestRegressor, RandomForestClassifier)):
        xprima = DMatrix(xprima)
    y_pred = model.predict(xprima)
    return y_pred


#Se entrena el modelo de regresión Ridge con las muestras perturbadas y los pesos calculados
def entrenar_modelo_ridge(R, Yprima, W):
    modelo_entrenado = Ridge(alpha=1.0) 
    modelo_entrenado.fit(R, Yprima, sample_weight=W)
    return modelo_entrenado

***MODELOS BICIS***

In [290]:
#RandomForest
#Creamos el modelo RandomForest (En este caso es Regressor ya que se trata de una tarea de regresión)
modelRandomForestBicis = RandomForestRegressor(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestBicis.fit(atributos_entrenamientoBicis, objetivo_entrenamientoBicis)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainBicis = xgb.DMatrix(atributos_entrenamientoBicis, label=objetivo_entrenamientoBicis)
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
dtestBicis2 = xgb.DMatrix(atributos_pruebaBicis)
# Definimos los parámetros del modelo XGBoost
paramsBicis = {
    'objective': 'reg:squarederror',  # Objetivo para regresión
    'eval_metric': 'rmse'  # Métrica de evaluación, error cuadrático medio (RMSE)
}

# Entrenamos el modelo XGBoost
modelXGBoostBicis = xgb.train(paramsBicis, dtrainBicis)

In [234]:
clon_model_XGboost_bicis = copy.deepcopy(modelXGBoostBicis)
prediccionesBicisXG = clon_model_XGboost_bicis.predict(dtestBicis)
explicacionesBicisXG = []
atributosAcambiar = 5
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model_XGboost_bicis,rangosBicis)
    explicacionesBicisXG.append(explicacion)
print(explicacionesBicisXG)

[array([[-1.41195934e+01, -1.37031956e-01, -1.34606524e-01,
        -1.36439576e+01, -2.07709812e-02, -4.14593183e-01,
         1.20675007e+01, -1.15183010e+01,  2.41789316e+00,
        -1.57249078e+01, -1.16260975e-01,  1.57116918e+01,
         2.50761831e+01,  5.56754700e-01]]), array([[ 4.81448922, -7.73947859, -0.05676592, -0.19900828, -0.0179591 ,
        -7.61848336,  6.36731746,  1.30793181,  3.65141952,  3.3030855 ,
        -7.7215195 , -3.30158561, -0.48415142,  7.69470826]]), array([[ 4.1245731 , -8.16238306, -0.13934561, -0.33344058, -0.0112054 ,
        -8.902275  ,  6.08655255,  2.95506806,  3.69124354,  1.85816827,
        -8.15117766, -1.85640794, -0.21395659,  9.05458634]]), array([[-3.91024522e+00, -7.24910088e+00, -1.10865001e-01,
         2.11081228e-01, -1.57718939e-02, -2.00529687e+01,
         1.56304541e+01,  4.53337963e+00,  3.66554848e-01,
         4.79594131e+00, -7.23332899e+00, -4.79449775e+00,
        -2.35168180e+00,  2.01810491e+01]]), array([[  3.4450054

In [235]:
prediccionesBicisRF = modelRandomForestBicis.predict(atributos_pruebaBicis)
explicacionesBicisRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestBicis,rangosBicis)
    explicacionesBicisRF.append(explicacion)
print(explicacionesBicisRF)

[array([[-1.27118748e+01, -1.29637436e-01, -1.27527645e-01,
        -1.36211722e+01, -1.95126966e-02, -4.10927417e-01,
         1.13021322e+01, -1.07636771e+01,  1.48804139e+00,
        -1.50576650e+01, -1.10124739e-01,  1.50453448e+01,
         2.45709531e+01,  5.45647569e-01]]), array([[ 2.61981055, -5.72533783, -0.05758607, -0.16102954, -0.0187367 ,
        -8.42872396,  7.09945913,  1.3868509 ,  1.27289238,  3.89317278,
        -5.70660113, -3.89183058, -0.78872886,  8.50638893]]), array([[  1.97101619,  -6.08644987,  -0.13631999,  -0.27128868,
         -0.01157593, -11.00532551,   8.04999067,   3.09165483,
         -0.34178963,   3.80285823,  -6.07487394,  -3.80122327,
         -0.3415295 ,  11.15485639]]), array([[ -1.92321982,  -7.3037724 ,  -0.14633366,  -1.50992509,
         -0.02004261, -13.14570042,  10.91436528,   2.3776688 ,
          3.99687663,   0.32735537,  -7.28372979,  -0.3255685 ,
          0.72816265,  13.31386356]]), array([[  2.56427985, -10.98442272,  -0.9514454

***VALIDACIÓN DE MODELO***

In [236]:
def evaluate_model(model, Xentrenamiento, yentrenamiento, Xtest, ytest):
    # Validación cruzada
    cv_scores = cross_val_score(model, Xentrenamiento, yentrenamiento, cv=2)
    mean_cv_score = np.mean(cv_scores)
    
    print("Puntuación media de validación cruzada:", mean_cv_score)
    
    #.score
    score = model.score(Xtest, ytest)
    print("Puntuación en el conjunto de prueba:", score)

In [237]:
evaluate_model(modelRandomForestBicis, atributos_entrenamientoBicis, objetivo_entrenamientoBicis, atributos_pruebaBicis, objetivo_pruebaBicis)

Puntuación media de validación cruzada: 0.9994299211430304
Puntuación en el conjunto de prueba: 0.9998270814120609


******

In [238]:


nuevos_ejemplos2 = pd.DataFrame(atributos_entrenamientoBicis).iloc[3]
dataframe = pd.DataFrame([nuevos_ejemplos2])
print(dataframe)


Iterac=10
atributosAcambiar=5

nuevos_ejemplos = pd.DataFrame([[1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7]])
#print("Prediccion RF",prediccion_de_modelo(dataframe,modelRandomForestBicis))
#print("Prediccion XG",prediccion_de_modelo(dataframe,modelXGBoostBicis))
#print("Método LIME con Random Forest: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelRandomForestBicis,rangosBicis))

#print("Método LIME con XGBoost: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelXGBoostBicis,rangosBicis))

    0    1    2     3    4    5    6    7     8       9     10     11    12  \
3  3.0  1.0  9.0  10.0  0.0  3.0  1.0  1.0  0.56  0.5303  0.52  0.194  23.0   

      13  
3  169.0  


******

***

***ACCESO A DATOS DE CSV PEN***

In [284]:

# Cargar el archivo CSV
pen = pd.read_csv('pendigits.csv', header=0, names=['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14','a15','a16',"class-objective"])

atributosPen = pen.loc[:, 'a1':'a16']
objetivoPen = pen['class-objective']

rangosPen = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in pen.columns:
    valor_min = pen[columna].min()
    valor_max = pen[columna].max()
    rangosPen[columna] = {'min': valor_min, 'max': valor_max}
    


In [285]:
(atributos_entrenamientoPen, atributos_pruebaPen,
 objetivo_entrenamientoPen, objetivo_pruebaPen) = model_selection.train_test_split(
    atributosPen, objetivoPen, test_size=.33,stratify=objetivoPen)

atributos_entrenamientoPen = atributos_entrenamientoPen.values
objetivo_entrenamientoPen = objetivo_entrenamientoPen.values
atributos_pruebaPen = atributos_pruebaPen.values
objetivo_pruebaPen= objetivo_pruebaPen.values

print(atributos_entrenamientoPen)

[[ 96  85  31 ...  62 100  91]
 [ 12  97  56 ...   0   0  12]
 [  0  55  38 ...  24  43   0]
 ...
 [ 78 100  23 ...  20   0   0]
 [ 23  71  41 ...   0   0  12]
 [ 86  62  21 ...  27  84   0]]


***MODELOS PEN***

In [286]:

modelRandomForestPen = RandomForestClassifier(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestPen.fit(atributos_entrenamientoPen, objetivo_entrenamientoPen)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainPen = xgb.DMatrix(atributos_entrenamientoPen, label=objetivo_entrenamientoPen)
dtestPen = xgb.DMatrix(atributos_pruebaPen)
dtestPen2 = xgb.DMatrix(atributos_pruebaPen)

# Definimos los parámetros del modelo XGBoost
paramsPen = {
    'objective': 'multi:softmax',  # Función objetivo para clasificación multiclase
    'num_class': 10,       # Número de clases en tu problema
    'eval_metric': 'merror' 
}

# Entrenamos el modelo XGBoost
modelXGBoostPen = xgb.train(paramsPen, dtrainPen)

In [273]:
clon_model_XGboost_pen = copy.deepcopy(modelXGBoostPen)
prediccionesPenXG = clon_model_XGboost_pen.predict(dtestPen)
explicacionesPenXG = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model_XGboost_pen, rangosPen)
    explicacionesPenXG.append(explicacion)
print(explicacionesPenXG)

[array([[ 0.01382707, -0.05285789, -0.04391987,  0.00262262, -0.01771307,
        -0.08251403,  0.22511738,  0.08747656, -0.01996488,  0.02569775,
        -0.03952481, -0.11368583,  0.00282583, -0.04690866, -0.00189473,
         0.06141659]]), array([[-0.05089117, -0.09597584, -0.01542408, -0.03048779, -0.1154    ,
        -0.01664862,  0.18403199, -0.00445766,  0.10067883,  0.11095876,
        -0.06006759, -0.07238675, -0.12488535,  0.06375776, -0.05553056,
         0.18272807]]), array([[-0.06891545,  0.02710205,  0.07772095,  0.28320568,  0.03098174,
        -0.13759344,  0.04804609, -0.31441114,  0.14662265,  0.02967877,
         0.03923669, -0.29355552, -0.13202385,  0.07752527,  0.16920401,
         0.01717552]]), array([[ 0.01705703,  0.0069428 ,  0.00167923,  0.00951706,  0.00954041,
         0.00814166, -0.02775925,  0.00802917, -0.01649205, -0.02650087,
         0.00944384,  0.01761715,  0.01873245, -0.01606648,  0.01132112,
        -0.03120327]]), array([[ 0.27528995, -0.178

In [243]:
prediccionesPenRF = modelRandomForestPen.predict(atributos_pruebaPen)
explicacionesPenRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestPen, rangosPen)
    explicacionesPenRF.append(explicacion)
print(explicacionesPenRF)

[array([[ 0.01043665,  0.00806897,  0.02294134, -0.02745302,  0.03580629,
         0.03834577,  0.05815203,  0.02138299, -0.03470404,  0.02725304,
        -0.03768969, -0.09441442,  0.05727865, -0.09988833, -0.0272749 ,
         0.04175868]]), array([[ 0.01870421,  0.00630032,  0.00227619,  0.01393315,  0.00893752,
         0.00841937, -0.03289509,  0.00383201, -0.01414165, -0.03093012,
         0.0122259 ,  0.02030926,  0.01728072, -0.01589379,  0.01573336,
        -0.03409137]]), array([[ 0.08350547, -0.15171722,  0.01301316,  0.0499631 , -0.12959528,
         0.02929421,  0.06568493,  0.02765737,  0.11919923, -0.12439119,
         0.04088573,  0.06815851, -0.10431359, -0.08253875,  0.05298265,
         0.04221167]]), array([[ 0.03821748,  0.00755222,  0.00630673,  0.02591187,  0.01888751,
         0.00769539, -0.04458448,  0.00803375, -0.0160043 , -0.05543098,
         0.0172135 ,  0.0303072 ,  0.02156103, -0.03165083,  0.02341039,
        -0.05742647]]), array([[-0.20579265, -0.093

In [244]:
nuevos_ejemplos2 = pd.DataFrame(atributos_entrenamientoPen).iloc[2]
dataframe = pd.DataFrame([nuevos_ejemplos2])
print(dataframe)
Iterac=10
atributosAcambiar=5


#print("Prediccion RF",prediccion_de_modelo(dataframe,modelRandomForestPen))
#print("Prediccion XG",prediccion_de_modelo(dataframe,modelXGBoostPen))
#print("Método LIME con Random Forest: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelRandomForestPen,rangosPen))
#print("Método LIME con XGBoost: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelXGBoostPen,rangosPen))

   0   1   2   3   4   5    6    7   8   9   10  11  12  13  14  15
2   0  44  26  62  62  81  100  100  97  78  79  52  64  26  53   0


***VALIDACIÓN***

In [245]:
evaluate_model(modelRandomForestPen, atributos_entrenamientoPen, objetivo_entrenamientoPen, atributos_pruebaPen, objetivo_pruebaPen)

Puntuación media de validación cruzada: 0.9844621513944223
Puntuación en el conjunto de prueba: 0.9890820865345734


In [298]:
explicaciones_alteradas_RF_Bicis=[]
explicaciones_alteradas_RF_Pen=[]
explicaciones_alteradas_XG_Bicis=[]
explicaciones_alteradas_XG_Pen=[]

def eliminar_irrelevantes_RF(pesos, k, muestra, modelo,rangos):
    array = np.array(pesos).flatten()
    pesos_abs = np.abs(array)
    indices = np.argsort(pesos_abs)[:k]
    copia = muestra.copy()
    for i in range(len(muestra)):
        if i in indices:
            copia[i] = 0
    muestra_2d = copia.reshape(1, -1)
    prediccion_nueva = modelo.predict(muestra_2d)
    dataframe = pd.DataFrame(muestra_2d)
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelo, rangos)
    if rangos == rangosBicis:
        explicaciones_alteradas_RF_Bicis.append(explicacion)
    else:
        explicaciones_alteradas_RF_Pen.append(explicacion)
    return prediccion_nueva



def eliminar_irrelevantes_XG(pesos, k, muestra, modelo,rangos):
    clon_model = copy.deepcopy(modelo)
    array = np.array(pesos).flatten()
    pesos_abs = np.abs(array)
    indices = np.argsort(pesos_abs)[:k]
    
    data = muestra.get_data()
    data_copy = data.copy()
    label = muestra.get_label()
    weight = muestra.get_weight()
    base_margin = muestra.get_base_margin()
    feature_names = muestra.feature_names
    # Copiar los valores y crear una nueva muestra
    
    for i in indices:
        data_copy[0,i]=0
    copia = xgb.DMatrix(data_copy, label=label, weight=weight, base_margin=base_margin)
    dataframe = pd.DataFrame([copia.get_data().data], columns=feature_names)
    prediccion_nueva = clon_model.predict(copia)
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model, rangos)
    if rangos == rangosBicis:
        explicaciones_alteradas_XG_Bicis.append(explicacion)
    else:
        explicaciones_alteradas_XG_Pen.append(explicacion)
    return prediccion_nueva




In [258]:
predicciones_RF_Bicis = prediccionesBicisRF[:256]
objetivos_bicis = objetivo_pruebaBicis[:256]
explicaciones_RF_Bicis = explicacionesBicisRF

predicciones_XG_Bicis = prediccionesBicisXG[:256]
explicaciones_XG_Bicis = explicacionesBicisXG

predicciones_RF_Pen = prediccionesPenRF[:256]
objetivos_pen = objetivo_pruebaPen[:256]
explicaciones_RF_Pen = explicacionesPenRF

predicciones_XG_Pen = prediccionesPenXG[:256]
explicaciones_XG_Pen = explicacionesPenXG

In [300]:
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
dtestPen = xgb.DMatrix(atributos_pruebaPen)
predicciones_alteradas_RF_Bicis=[]
predicciones_alteradas_RF_Pen=[]
predicciones_alteradas_XG_Bicis=[]
predicciones_alteradas_XG_Pen=[]

for i in range(256):
    
    pred = eliminar_irrelevantes_RF(explicaciones_RF_Bicis[i],5,atributos_pruebaBicis[i],modelRandomForestBicis,rangosBicis)
    predicciones_alteradas_RF_Bicis.append(pred)
    pred = eliminar_irrelevantes_RF(explicaciones_RF_Pen[i],5,atributos_pruebaPen[i],modelRandomForestPen,rangosPen)
    predicciones_alteradas_RF_Pen.append(pred)
    pred = eliminar_irrelevantes_XG(explicaciones_XG_Pen[i],5,dtestPen.slice([i]),modelXGBoostPen,rangosPen)
    predicciones_alteradas_XG_Pen.append(pred)
    pred = eliminar_irrelevantes_XG(explicaciones_XG_Bicis[i],5,dtestBicis.slice([i]),modelXGBoostBicis,rangosBicis)
    predicciones_alteradas_XG_Bicis.append(pred)
    
print(len(predicciones_alteradas_XG_Bicis))
print(len(predicciones_alteradas_XG_Pen))

256
256


***MÉTRICAS***

In [320]:
# Identidad

def identidad(i, muestra, lista_explicaciones):
    e = lista_explicaciones[i]
    return np.linalg.norm(muestra - muestra) != 0 or np.linalg.norm(e - e) == 0

# Separabilidad

def separabilidad(i,j,muestra_i,muestra_j, lista_explicaciones):
    e1 = lista_explicaciones[i]
    e2 = lista_explicaciones[j]
    return np.linalg.norm(muestra_i - muestra_j) == 0 or np.linalg.norm(e1 - e2) > 0 

# Estabilidad

def estabilidad(i, muestra, lista_muestras, lista_explicaciones):
    e = lista_explicaciones[i]
    d_muestras = np.array([np.linalg.norm(muestra - x) for x in lista_muestras])
    d_explicaciones = np.array([np.linalg.norm(e - x) for x in lista_explicaciones])
    return spearmanr(d_muestras, d_explicaciones)[0] > 0

# Selectividad

def selectividad(i):
    return null

# Coherencia
def coherencia(i, lista_predicciones, lista_objetivo,predicciones_actualizadas):
    error_prediccion_original = lista_predicciones[i] - lista_objetivo[i]
    error_prediccion_nuevo = predicciones_actualizadas[i] - lista_objetivo[i]
    return np.abs(error_prediccion_original - error_prediccion_nuevo)

# Completitud

def completitud(lista_predicciones, lista_objetivo,explicaciones,explicaciones_actualizadas):
    error_prediccion = np.mean(lista_predicciones - lista_objetivo)
    error_explicacion = np.mean([explicaciones[i] - explicaciones_actualizadas[i] for i in range(256)])
    return error_explicacion / error_prediccion

# Congruencia
def congruencia(lista_predicciones, lista_objetivo,predicciones_actualizadas):
    sumatorio_coherencia = 0
    sumatorio_desviacion = 0
    for i in range(len(lista_predicciones)):
        alpha = coherencia(i,lista_predicciones, lista_objetivo,predicciones_actualizadas)
        sumatorio_coherencia += alpha
    coherencia_promedio = sumatorio_coherencia / len(lista_predicciones)
    for i in range(len(lista_predicciones)):
        alpha = coherencia(i,lista_predicciones, lista_objetivo,predicciones_actualizadas)
        sumatorio_desviacion += np.power(alpha-coherencia_promedio,2)
    arg = sumatorio_desviacion / len(lista_predicciones)
    return np.sqrt(arg)

In [323]:
# Bicis Random Forest

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,atributos_pruebaBicis[i],explicaciones_RF_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,atributos_pruebaBicis[i],atributos_pruebaBicis[255],explicaciones_RF_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,atributos_pruebaBicis[i],atributos_pruebaBicis[:256],explicaciones_RF_Bicis)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_RF_Bicis,objetivos_bicis,predicciones_alteradas_RF_Bicis)[0]
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = completitud(predicciones_RF_Bicis,objetivos_bicis,explicaciones_RF_Bicis,explicaciones_alteradas_RF_Bicis)
print("\nCompletitud:", res_completitud)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_RF_Bicis, objetivos_bicis,predicciones_alteradas_RF_Bicis)[0])


Identidad: True

Separabilidad: True

Estabilidad: False

Coherencia: 191.21761718750008

Completitud: 1.2600606743833484e-16

Congruencia: 166.2109368375027


In [338]:
# Bicis XGBoost

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,dtestBicis.slice([i]).get_data().data,explicaciones_XG_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,dtestBicis.slice([i]).get_data().data,dtestBicis.slice([255]).get_data().data,explicaciones_XG_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

#res_estabilidad = True
#for i in range(0, 256):
#    res_estabilidad &= estabilidad(i,dtestBicis.slice([i]).get_data().data,dtestBicis.slice([0,256]).get_data().data,explicaciones_XG_Bicis)
#print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_XG_Bicis,objetivos_bicis,predicciones_alteradas_XG_Bicis)[0]
print("\nCoherencia:", res_coherencia/256)

# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_XG_Bicis,objetivos_bicis,predicciones_alteradas_XG_Bicis)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_XG_Bicis, objetivos_bicis,predicciones_alteradas_XG_Bicis)[0])


Identidad: True

Separabilidad: True

Coherencia: [185.57823]

Completitud: 2.181385915086139

Congruencia: [162.76822]


In [342]:
# Pen Random Forest

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,atributos_pruebaPen[i],explicaciones_RF_Pen)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,atributos_pruebaPen[i],atributos_pruebaPen[255],explicaciones_RF_Pen)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,atributos_pruebaPen[i],atributos_pruebaPen[:256],explicaciones_RF_Pen)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_RF_Pen,objetivos_pen,predicciones_alteradas_RF_Pen)[0]
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_RF_Pen,objetivos_pen,predicciones_alteradas_RF_Pen)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_RF_Pen, objetivos_pen,predicciones_alteradas_RF_Pen)[0])


Identidad: True

Separabilidad: True

Estabilidad: False

Coherencia: 3.296875

Completitud: 0.006175952653674274

Congruencia: 2.4760079229225016


In [341]:
# Pen XGBoost

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,dtestPen.slice([i]).get_data().data,explicaciones_RF_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,dtestPen.slice([i]).get_data().data,dtestPen.slice([255]).get_data().data,explicaciones_RF_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

#res_estabilidad = True
#lista_pen = [x for x in dtestPen[:256]]
#for i in range(0, 256):
#    res_estabilidad &= estabilidad(i,dtestPen.slice([i]).get_data(),lista_pen,explicaciones_RF_Bicis)
#print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_XG_Pen,objetivos_pen,predicciones_alteradas_XG_Pen)[0]
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_XG_Pen,objetivos_pen,predicciones_alteradas_XG_Pen)
print("\nCompletitud", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_XG_Pen, objetivos_pen,predicciones_alteradas_XG_Pen)[0])


Identidad: True

Separabilidad: True

Coherencia: [3.4726562]

Completitud 0.01781374831989524

Congruencia: [2.3797696]
